In [28]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [29]:
# Setup paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [30]:
!pip install pickleshare

In [32]:
%cd D:\setup

D:\setup


In [33]:
# Setup paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [34]:
# Make the directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

In [35]:
# Uncompress Tar GZ Labelled Faces in the Wild Dataset
!tar -xf lfw.tgz

In [36]:
# Import uuid library to generate unique image names
import uuid

In [37]:
os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))

'data\\anchor\\5a1d9eab-4f55-11ef-9f4a-00155d1d2527.jpg'

In [38]:
# Move LFW Images to the following repository data/negative
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', directory)):
        EX_PATH = os.path.join('lfw', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

In [39]:
!pip install tensorflow

In [45]:
# Establish a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250px
    frame = frame[120:120+250,200:200+250, :]
    
    # Collect anchors 
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Create the unique file path 
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path 
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out positive image
        cv2.imwrite(imgname, frame)
    
    # Show image back to screen
    cv2.imshow('Image Collection', frame)
    
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()

In [43]:
def data_aug(img):
    data = []
    for i in range(9):
        img = tf.image.stateless_random_brightness(img, max_delta=0.02, seed=(1,2))
        img = tf.image.stateless_random_contrast(img, lower=0.6, upper=1, seed=(1,3))
        # img = tf.image.stateless_random_crop(img, size=(20,20,3), seed=(1,2))
        img = tf.image.stateless_random_flip_left_right(img, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_jpeg_quality(img, min_jpeg_quality=90, max_jpeg_quality=100, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_saturation(img, lower=0.9,upper=1, seed=(np.random.randint(100),np.random.randint(100)))
            
        data.append(img)
    
    return data

In [46]:
# Import tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [50]:
for file_name in os.listdir(os.path.join(ANC_PATH)):
    img_path = os.path.join(ANC_PATH, file_name)
    img = cv2.imread(img_path)
    augmented_images = data_aug(img) 
    
    for image in augmented_images:
        cv2.imwrite(os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1())), image.numpy())

In [48]:
for file_name in os.listdir(os.path.join(POS_PATH)):
    img_path = os.path.join(POS_PATH, file_name)
    img = cv2.imread(img_path)
    augmented_images = data_aug(img) 
    
    for image in augmented_images:
        cv2.imwrite(os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1())), image.numpy())

In [51]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'\*.jpg').take(3000)
positive = tf.data.Dataset.list_files(POS_PATH+'\*.jpg').take(3000)
negative = tf.data.Dataset.list_files(NEG_PATH+'\*.jpg').take(3000)

In [55]:
import os
import random
import shutil

# Path to the folder containing images
folder_path = ANC_PATH

# Number of images to keep
num_to_keep = 1000

# List all files in the folder
all_files = os.listdir(folder_path)

# Filter out only the image files (you can customize this to match your image types)
image_files = [file for file in all_files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]

# Check if there are more than the specified number of images
if len(image_files) > num_to_keep:
    # Randomly select images to keep
    files_to_keep = random.sample(image_files, num_to_keep)
    files_to_delete = set(image_files) - set(files_to_keep)

    # Prompt the user for confirmation before deleting files
    print(f"Number of images to delete: {len(files_to_delete)}")
    confirm = input("Are you sure you want to delete these files? (yes/no): ")

    if confirm.lower() == 'yes':
        # Delete the files that are not in the list to keep
        for file in files_to_delete:
            file_path = os.path.join(folder_path, file)
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        print("Deletion completed.")
    else:
        print("Deletion aborted.")
else:
    print(f"There are only {len(image_files)} images, which is less than or equal to the number to keep.")


Number of images to delete: 2000


Are you sure you want to delete these files? (yes/no):  yes


Deleted: data\anchor\00a43857-4f58-11ef-be24-00155d1d2527.jpg
Deleted: data\anchor\f2daa6ff-4f57-11ef-9c00-00155d1d2527.jpg
Deleted: data\anchor\0792701e-4f58-11ef-833e-00155d1d2527.jpg
Deleted: data\anchor\f2f003c5-4f57-11ef-819e-00155d1d2527.jpg
Deleted: data\anchor\001d7358-4f58-11ef-8dda-00155d1d2527.jpg
Deleted: data\anchor\fd46d1f3-4f57-11ef-a2f1-00155d1d2527.jpg
Deleted: data\anchor\137141b8-4f58-11ef-8461-00155d1d2527.jpg
Deleted: data\anchor\004fe693-4f58-11ef-8181-00155d1d2527.jpg
Deleted: data\anchor\0603ad03-4f58-11ef-9e53-00155d1d2527.jpg
Deleted: data\anchor\00a32a42-4f58-11ef-bf76-00155d1d2527.jpg
Deleted: data\anchor\fac8c826-4f57-11ef-a005-00155d1d2527.jpg
Deleted: data\anchor\14cae98a-4f58-11ef-b75c-00155d1d2527.jpg
Deleted: data\anchor\f4f995db-4f57-11ef-8ed6-00155d1d2527.jpg
Deleted: data\anchor\0463dd51-4f58-11ef-a3bd-00155d1d2527.jpg
Deleted: data\anchor\08e0061e-4f58-11ef-a61e-00155d1d2527.jpg
Deleted: data\anchor\f7ce05bb-4f57-11ef-9323-00155d1d2527.jpg
Deleted:

In [56]:
import os
import random
import shutil

# Path to the folder containing images
folder_path = POS_PATH

# Number of images to keep
num_to_keep = 1000

# List all files in the folder
all_files = os.listdir(folder_path)

# Filter out only the image files (you can customize this to match your image types)
image_files = [file for file in all_files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]

# Check if there are more than the specified number of images
if len(image_files) > num_to_keep:
    # Randomly select images to keep
    files_to_keep = random.sample(image_files, num_to_keep)
    files_to_delete = set(image_files) - set(files_to_keep)

    # Prompt the user for confirmation before deleting files
    print(f"Number of images to delete: {len(files_to_delete)}")
    confirm = input("Are you sure you want to delete these files? (yes/no): ")

    if confirm.lower() == 'yes':
        # Delete the files that are not in the list to keep
        for file in files_to_delete:
            file_path = os.path.join(folder_path, file)
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        print("Deletion completed.")
    else:
        print("Deletion aborted.")
else:
    print(f"There are only {len(image_files)} images, which is less than or equal to the number to keep.")


Number of images to delete: 2000


Are you sure you want to delete these files? (yes/no):  yes


Deleted: data\positive\bd9490aa-4f57-11ef-9f16-00155d1d2527.jpg
Deleted: data\positive\d7c519f1-4f57-11ef-862d-00155d1d2527.jpg
Deleted: data\positive\c485f16e-4f57-11ef-81fb-00155d1d2527.jpg
Deleted: data\positive\b4bc6ed9-4f57-11ef-8f38-00155d1d2527.jpg
Deleted: data\positive\c7d2b8a6-4f57-11ef-8494-00155d1d2527.jpg
Deleted: data\positive\d485a5e1-4f57-11ef-986d-00155d1d2527.jpg
Deleted: data\positive\ce80a83d-4f57-11ef-bb08-00155d1d2527.jpg
Deleted: data\positive\c462fd36-4f57-11ef-9c3e-00155d1d2527.jpg
Deleted: data\positive\ad7935c7-4f57-11ef-8d24-00155d1d2527.jpg
Deleted: data\positive\ac6a85ed-4f57-11ef-a13a-00155d1d2527.jpg
Deleted: data\positive\ad6787e8-4f57-11ef-9725-00155d1d2527.jpg
Deleted: data\positive\c9de6de1-4f57-11ef-9edf-00155d1d2527.jpg
Deleted: data\positive\c744a949-4f57-11ef-b882-00155d1d2527.jpg
Deleted: data\positive\d0db98a2-4f57-11ef-98c6-00155d1d2527.jpg
Deleted: data\positive\49229019-4f56-11ef-88cb-00155d1d2527.jpg
Deleted: data\positive\4af2fedf-4f56-11e

In [57]:
import os
import random
import shutil

# Path to the folder containing images
folder_path = NEG_PATH

# Number of images to keep
num_to_keep = 1000

# List all files in the folder
all_files = os.listdir(folder_path)

# Filter out only the image files (you can customize this to match your image types)
image_files = [file for file in all_files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]

# Check if there are more than the specified number of images
if len(image_files) > num_to_keep:
    # Randomly select images to keep
    files_to_keep = random.sample(image_files, num_to_keep)
    files_to_delete = set(image_files) - set(files_to_keep)

    # Prompt the user for confirmation before deleting files
    print(f"Number of images to delete: {len(files_to_delete)}")
    confirm = input("Are you sure you want to delete these files? (yes/no): ")

    if confirm.lower() == 'yes':
        # Delete the files that are not in the list to keep
        for file in files_to_delete:
            file_path = os.path.join(folder_path, file)
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        print("Deletion completed.")
    else:
        print("Deletion aborted.")
else:
    print(f"There are only {len(image_files)} images, which is less than or equal to the number to keep.")


Number of images to delete: 2000


Are you sure you want to delete these files? (yes/no):  yes


Deleted: data\negative\Ivana_Trump_0001.jpg
Deleted: data\negative\Bill_Guerin_0001.jpg
Deleted: data\negative\Odilia_Collazo_0001.jpg
Deleted: data\negative\James_Layug_0001.jpg
Deleted: data\negative\Ari_Fleischer_0009.jpg
Deleted: data\negative\Mike_Weir_0004.jpg
Deleted: data\negative\Greg_Rusedski_0002.jpg
Deleted: data\negative\Mike_Weir_0010.jpg
Deleted: data\negative\Demi_Moore_0004.jpg
Deleted: data\negative\Jesse_James_Leija_0002.jpg
Deleted: data\negative\Eva_Dimas_0002.jpg
Deleted: data\negative\Gerhard_Schroeder_0045.jpg
Deleted: data\negative\Serena_Williams_0023.jpg
Deleted: data\negative\Jack_Grubman_0001.jpg
Deleted: data\negative\Joel_Gallen_0001.jpg
Deleted: data\negative\Gary_Coleman_0001.jpg
Deleted: data\negative\Alexandra_Vodjanikova_0001.jpg
Deleted: data\negative\Jennifer_Aniston_0015.jpg
Deleted: data\negative\Victoria_Clarke_0001.jpg
Deleted: data\negative\Tamika_Catchings_0001.jpg
Deleted: data\negative\Tyrone_Medley_0001.jpg
Deleted: data\negative\Calista_F